In [ ]:
import xarray as xr
import holoplot.pandas
import holoplot.xarray
import geoviews as gv
import cartopy.crs as ccrs

from bokeh.sampledata.airport_routes import airports

## Installation

The plot API also has support for geographic data built on top of Cartopy and GeoViews. Both can be installed using conda with:

    conda install -c pyviz geoviews
    
or if the cartopy dependency has been satisfied in some other way, GeoViews may also be installed using pip:

    pip install geoviews

## Usage

To declare a geographic plot we have to supply a ``cartopy.crs.CRS`` (or coordinate reference system).  Coordinate reference systems are described in the [GeoViews documentation](http://geoviews.org/user_guide/Projections.html) and the full list of available CRSs is in the [cartopy documentation](https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html). Only certain HoloPlot types support geographic coordinates, currently including: 'points', 'image', 'quadmesh', 'contour' and 'contourf'.

As an initial example, consider a dataframe of all US airports (including military bases overseas):

In [ ]:
airports.head(3)

### Declaring a coordinate system

To plot this data in a geographic context a ``crs`` can be supplied, declaring the coordinate system of the data coordinates. The default CRS for longitudes and latitudes is the ``PlateCarree`` projection. Once the ``crs`` is declared the data can be overlaid on top of ``geoviews.tile_sources`` and ``geoviews.features``:

In [ ]:
gv.tile_sources.ESRI * airports.holoplot.points(
    'Longitude', 'Latitude', crs=ccrs.PlateCarree(), color='red', alpha=0.2, height=500,
    xlim=(-180, -30), ylim=(0, 72)
)

### Declaring a projection

In addition to supplying a ``crs`` to declare the coordinate system in which the data points are stored, it is also possible to declare how to display the data on a custom ``projection``. After loading the same temperature dataset explored in the [Gridded Data](Gridded_Data.ipynb) section, the data can be displayed on an Orthographic projection:

In [ ]:
air_ds = xr.tutorial.load_dataset('air_temperature')

air_ds.holoplot.quadmesh(
    'lon', 'lat', 'air', crs=ccrs.PlateCarree(), projection=ccrs.Orthographic(-90, 30),
    global_extent=True, width=600, height=540, cmap='viridis'
) * gv.feature.coastline

Note that when displaying the data in a projection other than the one in which the data is stored, it is usually preferable to render it as a ``quadmesh`` rather than an ``image``. As you can see above, a QuadMesh will project each original bin or pixel into the correct non-rectangular shape, accurately showing the geographic extent covered by each sample. An Image, on the other hand, will always be rectangularly aligned in the 2D plan, which requires warping and resampling the data in a way that allows efficient display but loses accuracy at the pixel level.  Rendering a large QuadMesh using Bokeh can be very slow, but there are two useful alternatives for datasets too large to be practical as native QuadMeshes.

The first is using the ``datashade`` or ``rasterize`` options to regrid the data before rendering it, i.e., rendering the data on the backend and then sending a more efficient image-based representation to the browser:

In [ ]:
rasm = xr.tutorial.load_dataset('rasm')

rasm.holoplot.quadmesh(
    'xc', 'yc', crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    ylim=(0, 90), width=800, height=400, cmap='viridis', rasterize=True 
) * gv.feature.coastline

Another option that's still relatively slow but avoids sending large data into your browser is to plot the data using ``contour`` and ``contourf`` visualizations, generating a line or filled contour with a discrete number of levels:

In [ ]:
rasm.holoplot.contourf(
    'xc', 'yc', crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    ylim=(0, 90), width=800, height=400, cmap='viridis', levels=10, 
) * gv.feature.coastline

As you can see, HoloPlot makes it simple to work with geographic data visually.  For more complex plot types and additional details, see the [GeoViews](http://geoviews.org) documentation.